# Demos: Lecture 23

In [ ]:
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt

## Demo 1: depolarizing noise

In [ ]:
dev = qml.device("default.mixed", wires=1)

def prepare_state():
    qml.RY(2 * np.pi/3, wires=0)

@qml.qnode(dev)
def circuit_depolarizing(p):
    prepare_state()
    qml.DepolarizingChannel(p, wires=0)
    return qml.probs()

In [ ]:
circuit_depolarizing(0)

In [ ]:
circuit_depolarizing(0.02)

In [ ]:
@qml.qnode(dev)
def circuit_depolarizing(p):
    prepare_state()
    qml.DepolarizingChannel(p, wires=0)
    return qml.state()

In [ ]:
qml.math.fidelity(circuit_depolarizing(0), circuit_depolarizing(0.05))

In [ ]:
p_values = np.linspace(0., 0.75, 50)

plt.scatter(
    p_values,
    [qml.math.fidelity(circuit_depolarizing(0), circuit_depolarizing(p)) for p in p_values]
)

In [ ]:
qml.math.fidelity(circuit_depolarizing(0), np.eye(2)/2)

## Demo 2: Amplitude damping channel

In [ ]:
@qml.qnode(dev)
def circuit_ampdamp(p):
    prepare_state()
    qml.AmplitudeDamping(p, wires=0)
    return qml.probs()

In [ ]:
p_values = np.linspace(0., 1, 50)

plt.scatter(
    p_values,
    [circuit_ampdamp(p)[1] for p in p_values]
)

## Demo 3: simulated noisy VQE for deuteron

In [ ]:
coeffs = [28.657, 0.218, -6.125, -9.625, -13.125, -2.143, -3.913, -5.671, -2.143, -3.913, -5.671]

ops = [
    qml.Identity(0), 
    qml.PauliZ(0),
    qml.PauliZ(1),
    qml.PauliZ(2),
    qml.PauliZ(3),
    qml.PauliX(0) @ qml.PauliX(1),
    qml.PauliX(1) @ qml.PauliX(2),
    qml.PauliX(2) @ qml.PauliX(3),
    qml.PauliY(0) @ qml.PauliY(1),
    qml.PauliY(1) @ qml.PauliY(2),
    qml.PauliY(2) @ qml.PauliY(3)
]

H = qml.Hamiltonian(coeffs, ops, grouping_type="qwc")

In [ ]:
def ansatz(params):
    qml.PauliX(wires=0)
    for wire in range(3):
        qml.CRY(params[wire], wires=[wire, wire+1])
        qml.CNOT(wires=[wire+1, wire])
    return qml.expval(H)

def noise_model(p_dep, p_ampdamp, wires):
    qml.DepolarizingChannel(p_dep, wires=wires)
    qml.AmplitudeDamping(p_ampdamp, wires=wires)

In [ ]:
dev = qml.device("default.mixed", wires=4, shots=10000)
dev_noisy = qml.transforms.insert(dev, noise_model, [0.005, 0.001])

In [ ]:
normal_qnode = qml.QNode(ansatz, dev, diff_method="parameter-shift")
noisy_qnode = qml.QNode(ansatz, dev_noisy, diff_method="parameter-shift")

In [ ]:
normal_opt = qml.GradientDescentOptimizer(stepsize=0.05)
noisy_opt = qml.GradientDescentOptimizer(stepsize=0.05)

normal_params = np.zeros(3, requires_grad=True)
noisy_params = np.zeros(3, requires_grad=True)

normal_energies = []
noisy_energies = []

for idx in range(100):
    normal_params, normal_cost = normal_opt.step_and_cost(normal_qnode, normal_params)
    normal_energies.append(normal_cost)
    
    noisy_params, noisy_cost = noisy_opt.step_and_cost(noisy_qnode, noisy_params)
    noisy_energies.append(noisy_cost)
    
    if idx % 10 == 0:
        print(idx)

In [ ]:
plt.figure(figsize=(10, 7))
plt.scatter(range(100), normal_energies, label="Shot noise")
plt.scatter(range(100), noisy_energies, label="Hardware noise")
plt.axhline(y=-2.143, linestyle="--", label="True value", color="black")
plt.ylabel("Iteration", fontsize=16)
plt.yticks(fontsize=12)
plt.xlabel("Energy", fontsize=16)
plt.xticks(fontsize=12)
plt.legend(fontsize=16)